In [29]:
import pandas as pd
from sqlalchemy import create_engine, text
engine = create_engine('postgresql://postgres@db:5432/postgres')


In [ ]:
# query = """
# SELECT sr.execution_time, sr.success, fr.result_date
# FROM backend_step_result sr
# JOIN backend_feature_result fr 
#   ON fr.feature_result_id = sr.feature_result_id
# WHERE fr.feature_id_id = 1062
# LIMIT 100
# ORDER BY fr.result_date DESC
# ;
# """

In [6]:

query = "SELECT fr.feature_result_id, fr.success FROM backend_feature_result fr where fr.feature_id_id = 1062 and fr.success = 'True'"
df = pd.read_sql(query, engine)
df

,feature_result_id,success
0,437814,True
1,437831,True
2,437834,True
3,437853,True
4,437862,True
...,...,...
6334,456735,True
6335,456738,True
6336,456739,True
6337,456745,True


In [13]:

query = """ SELECT * from backend_step_result where feature_result_id = 456746 order by step_result_id"""

df = pd.read_sql(query, engine)
df

,step_result_id,feature_result_id,step_name,execution_time,pixel_diff,success,template_name,status,files,belongs_to,...,screenshot_difference,screenshot_style,screenshot_template,error,rest_api_id,notes,relative_execution_time,current_step_variables_value,database_query_result,step_type
0,5411247,456746,Maximize the browser,23,0,True,,Success,[],1062,...,,,,None,None,{},23,None,None,BROWSER
1,5411248,456746,# PRE-PROD #,5,0,True,,Success,[],1057,...,,,,None,None,{},28,None,None,BROWSER
2,5411249,456746,"StartBrowser and call URL ""$crisy_pro_pre_home...",1642,0,True,,Success,[],1057,...,,,,None,None,{},1670,None,None,BROWSER
3,5411250,456746,# sleep in order to stabalize the login page,4,0,True,,Success,[],1057,...,,,,None,None,{},1674,None,None,BROWSER
4,5411251,456746,"I sleep ""2"" seconds",2007,0,True,,Success,[],1057,...,,,,None,None,{},3681,None,None,BROWSER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,5411333,456746,"wait until I can see ""EUR"" on page",5368,0,True,,Success,[],1066,...,,,,None,None,{},100173,None,None,BROWSER
87,5411334,456746,"I move mouse to ""//span[@title=""Details""]"" and...",624,0,True,,Success,[],1066,...,,,,None,None,{},100797,None,None,BROWSER
88,5411335,456746,I can switch to new Window,7,0,True,,Success,[],1066,...,,,,None,None,{},100804,None,None,BROWSER
89,5411336,456746,"I can switch to iFrame with id ""application-tr...",1300,0,True,,Success,[],1066,...,,,,None,None,{},102104,None,None,BROWSER


In [25]:


query = """
Select * from (
    SELECT 
        fr.feature_result_id, 
        COUNT(sr.step_result_id) AS step_count
    FROM 
        backend_feature_result fr
    INNER JOIN 
        backend_step_result sr 
        ON fr.feature_result_id = sr.feature_result_id
    WHERE 
        fr.feature_id_id = 1062 
        AND fr.success = 'True'
    GROUP BY 
        fr.feature_result_id, fr.result_date
    ORDER BY 
        fr.result_date
    ) as result_summary where step_count > 25;
"""

df = pd.read_sql(query, engine)
df


# connection = engine.connect()
# result = connection.execute(text(query))
# rows = result.fetchall()  # <- store all rows in 'rows'
# len(rows)

,feature_result_id,step_count
0,412765,41
1,412771,42
2,436820,29
3,436828,35
4,436829,49
...,...,...
2817,456735,91
2818,456738,91
2819,456739,91
2820,456745,91


In [26]:
# Indexing starts from 1 
# Check for step results which belongs to given feature result
# Then check for the step result at what index this steps comes 
# # This can be checked here or can be received this index from frontend

# query = """ 
#     SELECT 
#         step_result_id,
#         ROW_NUMBER() OVER (ORDER BY step_result_id) AS step_index
#     FROM backend_step_result
#     WHERE feature_result_id = 456746;
# """

# query = "SELECT fr.feature_result_id FROM backend_feature_result fr LEFT JOIN backend_feature f ON fr.feature_id_id = f.feature_id where f.feature_id = 1062"

# query = "SELECT feature_result_id FROM backend_feature_result where feature_id_id = 1062 order by result_date;"


query = """ SELECT step_index
FROM (
    SELECT 
        step_result_id,
        ROW_NUMBER() OVER (ORDER BY step_result_id) AS step_index
    FROM backend_step_result
    WHERE feature_result_id = 456746
) AS indexed_steps
WHERE step_result_id = 5411333;
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    row = result.fetchone()
    step_index = row[0] if row else None
    
# print(step_index)

# df = pd.read_sql(query, engine)
# str(df["step_index"])



# with engine.connect() as connection:
#     result = connection.execute(text(query))
#     rows = result.fetchall()  # <- store all rows in 'rows'
#     for row in rows:
#         # print(row[0])  # print the first column of each row
#         feature_result_id = row[0]


#         query = f""" SELECT * FROM (
#                 SELECT *, ROW_NUMBER() OVER (ORDER BY step_result_id) AS step_index
#                 FROM backend_step_result
#                 WHERE feature_result_id = {feature_result_id}
#             ) AS indexed_steps
#             WHERE step_index = {step_index};
#         """  
        
#         # query = """
#         #     SELECT ROW_NUMBER() OVER (ORDER BY step_result_id) AS step_index, step_result_id, feature_result_id, execution_time, success, status, error
#         #     FROM backend_step_result
#         #     WHERE feature_result_id = 456746;
#         # """
        
#         df = pd.read_sql(query, engine)
#         df

In [40]:
query = """ SELECT step_index
FROM (
    SELECT 
        step_result_id,
        ROW_NUMBER() OVER (ORDER BY step_result_id) AS step_index
    FROM backend_step_result
    WHERE feature_result_id = 456746
) AS indexed_steps
WHERE step_result_id = 5411333;
"""

connection = engine.connect()
result = connection.execute(text(query))
row = result.fetchone()
step_index = row[0] if row else None
step_index
    

87

In [31]:


query = """SELECT feature_result_id FROM backend_feature_result WHERE feature_id_id = 1062 and success = 'True' and ORDER BY result_date;"""

connection = engine.connect()
result = connection.execute(text(query))
rows = result.fetchall()  # <- store all rows in 'rows'
len(rows)

6339

In [36]:

# for row in rows:
    # print(row[0])  # print the first column of each row
feature_result_id = rows[6330][0]
feature_result_id

456717

In [38]:
query = f""" SELECT * FROM (
        SELECT *, ROW_NUMBER() OVER (ORDER BY step_result_id) AS step_index
        FROM backend_step_result
        WHERE feature_result_id = {feature_result_id}
    ) AS indexed_steps
    WHERE step_index = {step_index};
"""

df = pd.read_sql(query, engine)
df

,step_result_id,feature_result_id,step_name,execution_time,pixel_diff,success,template_name,status,files,belongs_to,...,screenshot_style,screenshot_template,error,rest_api_id,notes,relative_execution_time,current_step_variables_value,database_query_result,step_type,step_index
0,5410133,456717,"wait until I can see ""EUR"" on page",5366,0,True,,Success,[],1066,...,,,None,None,{},95171,None,None,BROWSER,87


In [15]:
# For a given feature id and step_name it selects the given features feature_result and checkes for the given step and lists in the order by feature_results date

query = """ 
SELECT sr.*
FROM backend_step_result sr 
LEFT JOIN backend_feature_result fr 
  ON sr.feature_result_id = fr.feature_result_id 
WHERE fr.feature_id_id = 1062 AND sr.step_name = 'Press Enter' AND sr.success='True'
order by fr.result_date
;
"""

df = pd.read_sql(query, engine)
df

,step_result_id,feature_result_id,step_name,execution_time,pixel_diff,success,template_name,status,files,belongs_to,...,screenshot_difference,screenshot_style,screenshot_template,error,rest_api_id,notes,relative_execution_time,current_step_variables_value,database_query_result,step_type
0,4893754,412713,Press Enter,50,0,True,,Success,[],1057,...,,,,None,None,{},2887,{},[],BROWSER
1,4893751,412713,Press Enter,67,0,True,,Success,[],1057,...,,,,None,None,{},1761,{},[],BROWSER
2,4893761,412714,Press Enter,67,0,True,,Success,[],1057,...,,,,None,None,{},1433,{},[],BROWSER
3,4893764,412714,Press Enter,64,0,True,,Success,[],1057,...,,,,None,None,{},2569,{},[],BROWSER
4,4893863,412718,Press Enter,70,0,True,,Success,[],1057,...,,,,None,None,{},3495,{},[],BROWSER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48041,5411254,456746,Press Enter,39,0,True,,Success,[],1057,...,,,,None,None,{},5387,None,None,BROWSER
48042,5411257,456746,Press Enter,353,0,True,,Success,[],1057,...,,,,None,None,{},7859,None,None,BROWSER
48043,5411268,456746,Press Enter,22,0,True,,Success,[],1057,...,,,,None,None,{},26297,None,None,BROWSER
48044,5411331,456746,Press Enter,52,0,True,,Success,[],1066,...,,,,None,None,{},94803,None,None,BROWSER


In [46]:
# feature_index = 6331  # Python index + 1
# step_index = 5

query = f"""
WITH filtered_features AS (
    SELECT feature_result_id
    FROM backend_feature_result
    WHERE feature_id_id = 1062 AND success = 'True'
)
 SELECT ROW_NUMBER() OVER (PARTITION BY sr.feature_result_id ORDER BY sr.step_result_id) AS step_index, sr.*
    FROM backend_step_result sr INNER JOIN filtered_features ff ON sr.feature_result_id = ff.feature_result_id;
"""
df = pd.read_sql(query, engine)
df.head(100)

,step_index,step_result_id,feature_result_id,step_name,execution_time,pixel_diff,success,template_name,status,files,...,screenshot_difference,screenshot_style,screenshot_template,error,rest_api_id,notes,relative_execution_time,current_step_variables_value,database_query_result,step_type
0,1,4894570,412765,# QS4,7,0,True,,Success,[],...,,,,None,None,{},7,{},[],BROWSER
1,2,4894571,412765,"Save ""https://trucklog-crisy4dtag-int.launchpa...",69,0,True,,Success,[],...,,,,None,None,{},76,{},[],BROWSER
2,3,4894572,412765,"StartBrowser and call URL ""$CRISY_HOMEPAGE""",1028,0,True,,Success,[],...,,,,None,None,{},1104,{},[],BROWSER
3,4,4894573,412765,"wait until I can see ""Mercedes"" on page",17,0,True,,Success,[],...,,,,None,None,{},1121,{},[],BROWSER
4,5,4894574,412765,"Send keys ""// text is set in step description!!""",80,0,True,,Success,[],...,,,,None,None,{},1201,{},[],BROWSER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,13,4905848,414379,"I can switch to iFrame with id ""application-tr...",269,0,True,,Success,[],...,,,,None,None,{},4670,{},[],BROWSER
96,14,4905849,414379,"I sleep ""2"" seconds",2005,0,True,,Success,[],...,,,,None,None,{},6675,{},[],BROWSER
97,15,4905850,414379,"wait until I can see ""To start, set the releva...",4613,0,True,,Success,[],...,,,,None,None,{},11288,{},[],BROWSER
98,16,4905851,414379,"wait until I can see ""Company DAIMLER TRUCK AG...",6259,0,True,,Success,[],...,,,,None,None,{},17547,{},[],BROWSER


In [48]:
df[100:150]

,step_index,step_result_id,feature_result_id,step_name,execution_time,pixel_diff,success,template_name,status,files,...,screenshot_difference,screenshot_style,screenshot_template,error,rest_api_id,notes,relative_execution_time,current_step_variables_value,database_query_result,step_type
100,18,4905853,414379,"I move mouse to ""(//input[@class=""sapMInputBas...",697,0,True,,Success,[],...,,,,None,None,{},20250,{},[],BROWSER
101,19,4905854,414379,"Send keys ""// text is set in step description!!""",102,0,True,,Success,[],...,,,,None,None,{},20352,{},[],BROWSER
102,20,4905855,414379,Press Enter,111,0,True,,Success,[],...,,,,None,None,{},20463,{},[],BROWSER
103,21,4905856,414379,"I sleep ""15"" seconds",15021,0,True,,Success,[],...,,,,None,None,{},35484,{},[],BROWSER
104,22,4905857,414379,"I move mouse to ""//span[@title=""Details""]"" and...",1055,0,True,,Success,[],...,,,,None,None,{},36539,{},[],BROWSER
105,23,4905858,414379,I can switch to new Window,19,0,True,,Success,[],...,,,,None,None,{},36558,{},[],BROWSER
106,24,4905859,414379,"I can switch to iFrame with id ""application-tr...",1483,0,True,,Success,[],...,,,,None,None,{},38041,{},[],BROWSER
107,25,4905860,414379,"wait until I can see ""Company DAIMLER TRUCK AG...",16795,0,True,,Success,[],...,,,,None,None,{},54836,{},[],BROWSER
108,1,4906001,414422,# PRE-PROD,7,0,True,,Success,[],...,,,,None,None,{},7,{},[],BROWSER
109,2,4906002,414422,"StartBrowser and call URL ""$crisy_pre_prod""",566,0,True,,Success,[],...,,,,None,None,{},573,{},[],BROWSER


In [4]:


query = """
Select * from backend_step_result where feature result;
"""

df = pd.read_sql(query, engine)
df

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "result"
LINE 2: Select * from backend_step_result where feature result;
                                                        ^

[SQL: 
Select * from backend_step_result where feature result;
]
(Background on this error at: https://sqlalche.me/e/20/f405)